In [ ]:
!pip install kaggle


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


# **Getting commonly avaliabel Gtsrb dataset**


In [ ]:
!kaggle datasets download -d "meowmeowmeowmeowmeow/gtsrb-german-traffic-sign"

Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
100% 611M/612M [00:28<00:00, 24.1MB/s]
100% 612M/612M [00:28<00:00, 22.9MB/s]


In [ ]:
shutil.rmtree('/content/Train')

In [ ]:
!unzip "/content/gtsrb-german-traffic-sign.zip"

Streaming output truncated to the last 5000 lines.
  inflating: train/5/00005_00053_00010.png  
  inflating: train/5/00005_00053_00011.png  
  inflating: train/5/00005_00053_00012.png  
  inflating: train/5/00005_00053_00013.png  
  inflating: train/5/00005_00053_00014.png  
  inflating: train/5/00005_00053_00015.png  
  inflating: train/5/00005_00053_00016.png  
  inflating: train/5/00005_00053_00017.png  
  inflating: train/5/00005_00053_00018.png  
  inflating: train/5/00005_00053_00019.png  
  inflating: train/5/00005_00053_00020.png  
  inflating: train/5/00005_00053_00021.png  
  inflating: train/5/00005_00053_00022.png  
  inflating: train/5/00005_00053_00023.png  
  inflating: train/5/00005_00053_00024.png  
  inflating: train/5/00005_00053_00025.png  
  inflating: train/5/00005_00053_00026.png  
  inflating: train/5/00005_00053_00027.png  
  inflating: train/5/00005_00053_00028.png  
  inflating: train/5/00005_00053_00029.png  
  inflating: train/5/00005_00054_00000.png  
  in

Keeping only required classes

In [ ]:
import os
import pandas as pd
import shutil

# Define the mapping for old class IDs to new class IDs
class_mapping = {
    0: 1, 11: 2, 12: 3, 13: 4, 14: 5, 15: 6, 17: 7, 18: 8, 19: 9,
    20: 10, 21: 11, 22: 12, 23: 13, 24: 14, 25: 15, 26: 16, 27: 17,
    28: 18, 29: 19, 30: 20
}

# Load the CSV file
input_csv = "/content/Train.csv"
data = pd.read_csv(input_csv)

# Filter rows based on the allowed class IDs
allowed_classes = set(class_mapping.keys())
filtered_data = data[data["ClassId"].isin(allowed_classes)].copy()

# Replace old class IDs with new class IDs
filtered_data["ClassId"] = filtered_data["ClassId"].map(class_mapping)

# Update file paths with new class folders
def update_path(row):
    old_path = row["Path"]
    old_class_id = old_path.split('/')[1]  # Extract old class ID from path
    new_class_id = row["ClassId"]
    return old_path.replace(f"/{old_class_id}/", f"/{new_class_id}/")

filtered_data["Path"] = filtered_data.apply(update_path, axis=1)

# Save the updated CSV file
output_csv = "/content/updated_traffic_images.csv"
filtered_data.to_csv(output_csv, index=False)

# Rename folders and move images
for _, row in filtered_data.iterrows():
    old_path = row["Path"]
    new_path = old_path  # Updated path is already correct

    # Create the new folder if it doesn't exist
    new_folder = os.path.dirname(new_path)
    os.makedirs(new_folder, exist_ok=True)

    # Move the image file to the new folder
    if os.path.exists(old_path):
        shutil.move(old_path, new_path)

print(f"Dataset processed and saved to {output_csv}")


Dataset processed and saved to /content/updated_traffic_images.csv


In [ ]:
import os
import shutil

# Define the mapping for old class IDs to new class IDs
class_mapping = {
    0: 1, 11: 2, 12: 3, 13: 4, 14: 5, 15: 6, 17: 7, 18: 8, 19: 9,
    20: 10, 21: 11, 22: 12, 23: 13, 24: 14, 25: 15, 26: 16, 27: 17,
    28: 18, 29: 19, 30: 20
}


# Paths
old_train_folder = "/content/Train"
new_train_folder = "/content/TrainNew"

# Create the new train folder
os.makedirs(new_train_folder, exist_ok=True)

# Iterate through the old class folders
for old_class_id in os.listdir(old_train_folder):
    old_class_path = os.path.join(old_train_folder, old_class_id)

    # Skip if not a directory
    if not os.path.isdir(old_class_path):
        continue

    # Convert class ID to integer for mapping
    try:
        old_class_id = int(old_class_id)
    except ValueError:
        print(f"Skipping invalid folder: {old_class_id}")
        continue

    # Check if the class is in the mapping
    if old_class_id in class_mapping:
        new_class_id = class_mapping[old_class_id]
        new_class_folder = os.path.join(new_train_folder, str(new_class_id))

        # Create the new class folder if it doesn't exist
        os.makedirs(new_class_folder, exist_ok=True)

        # Move all images from old folder to new folder
        for image_file in os.listdir(old_class_path):
            old_image_path = os.path.join(old_class_path, image_file)
            new_image_path = os.path.join(new_class_folder, image_file)

            shutil.move(old_image_path, new_image_path)

        print(f"Moved class {old_class_id} to {new_class_id}")
    else:
        # Delete the folder if not in the mapping
        shutil.rmtree(old_class_path)
        print(f"Deleted class folder: {old_class_id}")

print(f"Images have been reorganized into {new_train_folder}")


Moved class 21 to 11
Moved class 19 to 9
Deleted class folder: 35
Deleted class folder: 31
Deleted class folder: 2
Deleted class folder: 34
Deleted class folder: 1
Deleted class folder: 32
Deleted class folder: 9
Moved class 11 to 2
Deleted class folder: 6
Deleted class folder: 4
Moved class 20 to 10
Moved class 12 to 3
Moved class 18 to 8
Moved class 0 to 1
Deleted class folder: 33
Moved class 30 to 20
Moved class 17 to 7
Moved class 26 to 16
Deleted class folder: 41
Moved class 23 to 13
Moved class 22 to 12
Moved class 13 to 4
Deleted class folder: 37
Deleted class folder: 7
Moved class 27 to 17
Deleted class folder: 10
Deleted class folder: 40
Moved class 15 to 6
Deleted class folder: 3
Deleted class folder: 39
Moved class 24 to 14
Deleted class folder: 5
Deleted class folder: 36
Moved class 28 to 18
Moved class 14 to 5
Deleted class folder: 8
Moved class 29 to 19
Deleted class folder: 16
Deleted class folder: 42
Deleted class folder: 38
Moved class 25 to 15
Images have been reorgan

In [ ]:
import shutil
# shutil.rmtree('/content/Train')

In [ ]:
import os
import pandas as pd
from xml.etree.ElementTree import Element, SubElement, tostring
import xml.dom.minidom

# Load the dataset CSV
input_csv = "/content/updated_traffic_images.csv"
data = pd.read_csv(input_csv)

# Define the output directory for Pascal VOC XML files
output_dir = "/content/Train/Annotations"
os.makedirs(output_dir, exist_ok=True)

def create_voc_annotation(row):
    """Create Pascal VOC XML annotation for a given image row."""
    # Create the root element
    annotation = Element('annotation')

    # Add folder and filename
    folder = SubElement(annotation, 'folder')
    folder.text = 'Train'

    filename = SubElement(annotation, 'filename')
    filename.text = os.path.basename(row['Path'])

    # Add path
    path = SubElement(annotation, 'path')
    path.text = os.path.join("/content", row['Path'])

    # Add source
    source = SubElement(annotation, 'source')
    database = SubElement(source, 'database')
    database.text = 'Unknown'

    # Add image size
    size = SubElement(annotation, 'size')
    width = SubElement(size, 'width')
    width.text = str(row['Width'])
    height = SubElement(size, 'height')
    height.text = str(row['Height'])
    depth = SubElement(size, 'depth')
    depth.text = '3'  # Assuming 3-channel images (RGB)

    # Add segmented flag
    segmented = SubElement(annotation, 'segmented')
    segmented.text = '0'

    # Add object details
    obj = SubElement(annotation, 'object')
    name = SubElement(obj, 'name')
    name.text = str(row['ClassId'])  # Use ClassId as the object class
    pose = SubElement(obj, 'pose')
    pose.text = 'Unspecified'
    truncated = SubElement(obj, 'truncated')
    truncated.text = '0'
    difficult = SubElement(obj, 'difficult')
    difficult.text = '0'

    bndbox = SubElement(obj, 'bndbox')
    xmin = SubElement(bndbox, 'xmin')
    xmin.text = str(row['Roi.X1'])
    ymin = SubElement(bndbox, 'ymin')
    ymin.text = str(row['Roi.Y1'])
    xmax = SubElement(bndbox, 'xmax')
    xmax.text = str(row['Roi.X2'])
    ymax = SubElement(bndbox, 'ymax')
    ymax.text = str(row['Roi.Y2'])

    # Convert to a pretty XML string
    xml_str = xml.dom.minidom.parseString(tostring(annotation)).toprettyxml(indent="  ")
    return xml_str

# Generate Pascal VOC XML files
for _, row in data.iterrows():
    # Create Pascal VOC XML for the row
    xml_content = create_voc_annotation(row)

    # Write the XML to a file
    output_file = os.path.join(output_dir, os.path.splitext(os.path.basename(row['Path']))[0] + ".xml")
    with open(output_file, 'w') as f:
        f.write(xml_content)

print(f"Pascal VOC annotations saved to {output_dir}")


Pascal VOC annotations saved to /content/Train/Annotations


In [ ]:
import shutil

shutil.rmtree('/content/Train')

In [ ]:
!pip install -q tensorflow tensorflow-object-detection-api pillow lxml matplotlib



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 744.4/744.4 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependenc

In [ ]:
!apt-get install -y protobuf-compiler python3-tk


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-tk is already the newest version (3.10.8-1~22.04).
protobuf-compiler is already the newest version (3.12.4-1ubuntu7.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import tensorflow as tf
import os
import glob
import random
import xml.etree.ElementTree as ET

def create_tf_example(xml_file, label_map):
    """Convert Pascal VOC XML file to TensorFlow Example."""
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Extract filename from XML
    filename = root.find('filename').text

    # Extract full image path from the <path> tag
    filepath = root.find('path').text

    # Check if the file exists
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Image file not found: {filepath}")

    # Image size
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)

    # Lists for bounding box data and classes
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for member in root.findall('object'):
        xmins.append(float(member.find('bndbox/xmin').text) / width)
        xmaxs.append(float(member.find('bndbox/xmax').text) / width)
        ymins.append(float(member.find('bndbox/ymin').text) / height)
        ymaxs.append(float(member.find('bndbox/ymax').text) / height)
        class_name = member.find('name').text
        classes_text.append(class_name.encode('utf8'))
        if class_name not in label_map:
            raise ValueError(f"Class '{class_name}' not found in label map.")
        classes.append(label_map[class_name])  # Map name to class ID

    # Read image file
    with tf.io.gfile.GFile(filepath, 'rb') as fid:
        encoded_jpg = fid.read()

    # Create TF Example
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_jpg])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'png'])),  # Assuming PNG format
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
    }))
    return tf_example


# Define label map (class names mapped to integers 1-20, with 0 reserved for background)
label_map = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    '10': 10,
    '11': 11,
    '12': 12,
    '13': 13,
    '14': 14,
    '15': 15,
    '16': 16,
    '17': 17,
    '18': 18,
    '19': 19,
    '20': 20,
}


# Path to Pascal VOC XML annotations
xml_files = glob.glob('/content/Train/Annotations/*.xml')  # All annotations in a single folder

# Shuffle the dataset and split into train and validation sets
random.shuffle(xml_files)
split_ratio = 0.8  # 80% training, 20% validation
train_files = xml_files[:int(len(xml_files) * split_ratio)]
val_files = xml_files[int(len(xml_files) * split_ratio):]

# Generate TFRecord for training
train_tfrecord_path = '/content/train.tfrecord'
with tf.io.TFRecordWriter(train_tfrecord_path) as writer:
    for xml_file in train_files:
        tf_example = create_tf_example(xml_file, label_map)
        writer.write(tf_example.SerializeToString())

print(f"Training TFRecord file created at {train_tfrecord_path}")

# Generate TFRecord for validation
val_tfrecord_path = '/content/val.tfrecord'
with tf.io.TFRecordWriter(val_tfrecord_path) as writer:
    for xml_file in val_files:
        tf_example = create_tf_example(xml_file, label_map)
        writer.write(tf_example.SerializeToString())

print(f"Validation TFRecord file created at {val_tfrecord_path}")


Training TFRecord file created at /content/train.tfrecord
Validation TFRecord file created at /content/val.tfrecord


In [ ]:
!pip install protobuf==3.20.3
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.26.1 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.


env: PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python


In [ ]:
import shutil
shutil.rmtree('/content/models')

In [ ]:
# Clone the tensorflow models repository from GitHub
!pip uninstall Cython -y # Temporary fix for "No module named 'object_detection'" error
!git clone --depth 1 https://github.com/tensorflow/models


Found existing installation: Cython 3.0.11
Uninstalling Cython-3.0.11:
  Successfully uninstalled Cython-3.0.11
Cloning into 'models'...
remote: Enumerating objects: 4305, done.
remote: Counting objects: 100% (4305/4305), done.
remote: Compressing objects: 100% (3323/3323), done.
remote: Total 4305 (delta 1207), reused 2092 (delta 909), pack-reused 0 (from 0)
Receiving objects: 100% (4305/4305), 53.17 MiB | 21.96 MiB/s, done.
Resolving deltas: 100% (1207/1207), done.


In [ ]:
# Copy setup files into models/research folder
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
#cp object_detection/packages/tf2/setup.py .

In [ ]:
# Modify setup.py file to install the tf-models-official repository targeted at TF v2.8.0
import re
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [ ]:
# Install the Object Detection API (NOTE: This block takes about 10 minutes to finish executing)

# Need to do a temporary fix with PyYAML because Colab isn't able to install PyYAML v5.4.1
!pip install pyyaml==5.3
!pip install /content/models/research/

# Need to downgrade to TF v2.8.0 due to Colab compatibility bug with TF v2.10 (as of 10/03/22)
!pip install tensorflow==2.8.0

# Install CUDA version 11.0 (to maintain compatibility with TF v2.8.0)
!pip install tensorflow_io==0.23.1
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!apt-get update && sudo apt-get install cuda-toolkit-11-0
!export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.2/268.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.3-cp310-cp310-linux_x86_64.whl size=44244 sha256=e7f3f3cd1220621bcf82dcc6e18f6febe949ebc77efa2101ef14c1df1d84ff8c
  Stored in directory: /root/.cache/pip/wheels/0d/72/68/a263cfc14175636cf26bada99f13b735be1b60a11318e08bfc
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.2
    Uninstalling PyYAML-6.0.2:
      Successfully uninstalled PyYAML-6.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2024.10.0 requires pyyaml>=5.3.1, but you have pyyaml 5.3 which is incompatible.
flax 0.10.2 requires PyYAML>=5.4.1, but you have pyyaml 5.3 which is incompatible.
roboflow 1.1.50 requires PyYAML>=5.3.1, but you have pyyaml 5.3 which is incompatible.


Processing ./models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could tak

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 36.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.1
    Uninstalling tensorflow-2.8.1:
      Successfully uninstalled tensorflow-2.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.37.1
    Uninstalling tensorflow-io-gcs-filesystem-0.37.1:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.37.1
  Attempting uninstall: tensorflow_io
    Found existing installation: tensorflow-io 0.37.1
    Uninstalling tensorflow-io-0.37.1:
      Successfully uninstalled tensorflow-io-0.37.1
--2025-01-11 15:49:16--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu18

In [ ]:
# Run Model Bulider Test file, just to verify everything's working properly
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2025-01-11 15:54:09.810542: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0111 15:54:10.109473 132691044171776 model_builder.py:1112] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 1.78s
I0111 15:54:10.649240 132691044171776 test_util.py:2373] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 1.78s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.57s
I0111 15:54:11.216013 132691044171776 tes

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="V9lggyHp9idDjdLHtQsR")
project = rf.workspace("ssd-0jqhr").project("sssd2")
version = project.version(1)
dataset = version.download("tfrecord")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to sssd2-1 in tfrecord:: 100%|██████████| 8/8 [00:00<00:00, 36.15it/s]


In [ ]:
train_record_fname = '/content/sssd2-1/train/train.tfrecord'
val_record_fname = '/content/sssd2-1/valid/val.tfrecord'
label_map_pbtxt_fname = '/content/sssd2-1/train/label_map.pbtxt'

In [ ]:
# Change the chosen_model variable to deploy different models available in the TF2 object detection zoo
chosen_model = 'ssd-mobilenet-v2-fpnlite-320'

MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
}

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [ ]:
import shutil

shutil.rmtree('/content/models/mymodel')

In [ ]:
# Create "mymodel" folder for holding pre-trained weights and configuration files
%mkdir /content/models/mymodel/
%cd /content/models/mymodel/

# Download pre-trained model weights
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/content/models/mymodel
--2025-01-11 15:55:31--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.174.207, 74.125.23.207, 74.125.203.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.174.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  10.1MB/s    in 1.9s    

2025-01-11 15:55:34 (10.1 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

--2025-01-11 15:55:34--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.

In [ ]:
# Set training parameters for the model
num_steps = 10000

batch_size = 16

# learning_rate_base = 0.08 * (batch_size / 8)

In [ ]:
# Set file locations and get number of classes for config file
pipeline_fname = '/content/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total classes:', num_classes)


Total classes: 20


In [ ]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

%cd /content/models/mymodel
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # # Enable evaluation during training
    # s = re.sub('num_steps: [0-9]+',
    #            'num_steps: {}'.format(num_steps), s)

    # # Set evaluation metrics
    # s = re.sub('metrics_set: ".*?"',
    #            'metrics_set: "{}"'.format('coco_detection_metrics'), s)

    # # Ensure model is using COCO metrics
    # s = re.sub('use_moving_averages: false', 'use_moving_averages: true', s)


    # If using ssd-mobilenet-v2, reduce learning rate (because it's too high in the default config file)
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)

/content/models/mymodel
writing custom configuration file


In [ ]:
# (Optional) Display the custom configuration file's contents
!cat /content/models/mymodel/pipeline_file.config

# SSD with Mobilenet v2 FPN-lite (go/fpn-lite) feature extractor, shared box
# predictor and focal loss (a mobile version of Retinanet).
# Retinanet: see Lin et al, https://arxiv.org/abs/1708.02002
# Trained on COCO, initialized from Imagenet classification checkpoint
# Train on TPU-8
#
# Achieves 22.2 mAP on COCO17 Val

model {
  ssd {
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    num_classes: 20
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    encode_background_as_zeros: true
    anchor_generator {
      multiscale_anchor_generator 

In [ ]:
import shutil
shutil.rmtree('/content/training/train')
shutil.rmtree('/content/training')

In [ ]:
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = '/content/models/mymodel/pipeline_file.config'
model_dir = '/content/training/'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/content/models/mymodel/training/"


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.

In [ ]:
# Run training!
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --output_directory=/content/models/mymodel/exported_model \
    --checkpoint_every_n=1000





/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're us

In [ ]:
# Make a directory to store the trained TFLite model
!mkdir /content/custom_model_lite
output_directory = '/content/custom_model_lite'

# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = '/content/training/'

!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

2025-01-11 16:53:16.702562: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0111 16:53:20.562964 138635910717440 api.py:441] feature_map_spatial_dims: [(40, 40), (20, 20), (10, 10), (5, 5), (3, 3)]
I0111 16:53:25.985420 138635910717440 api.py:441] feature_map_spatial_dims: [(40, 40), (20, 20), (10, 10), (5, 5), (3, 3)]
2025-01-11 16:53:27.650296: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
I0111 16:53:28.488965 138635910717440 api.py:441] feature_map_spatial_dims: [(40, 40), (20, 20), (10, 10), (5, 5), (3, 3)]
W0111 16:53:29.783619 138635910717440 save_impl.py:71] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x7e16194f09d0>, because it is not built.
W0111 16:53:30.030827

In [ ]:
# Convert exported graph file into TFLite model file
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/custom_model_lite/saved_model')
tflite_model = converter.convert()

with open('/content/custom_model_lite/detect.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Import packages
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

### Define preprocessing function for image normalization and resizing
def preprocess_image(image_path, height, width, float_input, input_mean=127.5, input_std=127.5):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image_rgb, (width, height))
    input_data = np.expand_dims(image_resized, axis=0)
    if float_input:
        input_data = (np.float32(input_data) - input_mean) / input_std
    return input_data, image_rgb

### Define function for inferencing with TFLite model and displaying results
def tflite_detect_images(modelpath, imgpath, lblpath, min_conf=0.1, num_test_images=10, savepath='/content/results', txt_only=False):

  # Grab filenames of all images in test folder
  images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

  # Load the label map into memory
  labels = []
  with open(lblpath, 'r') as f:
      for line in f:
          if "name:" in line:
              labels.append(line.split('"')[1])

  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  # Randomly select test images
  images_to_test = random.sample(images, num_test_images)

  # Loop over every image and perform detection
  for image_path in images_to_test:

      # Preprocess image
      input_data, image_rgb = preprocess_image(image_path, height, width, float_input)
      imH, imW, _ = image_rgb.shape

      # Perform the actual detection by running the model with the image as input
      interpreter.set_tensor(input_details[0]['index'], input_data)
      interpreter.invoke()

      # Retrieve detection results
      boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
      classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
      scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

      detections = []

      # Loop over all detections and draw detection box if confidence is above minimum threshold
      for i in range(len(scores)):
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

              # Get bounding box coordinates and draw box
              ymin = int(max(1, (boxes[i][0] * imH)))
              xmin = int(max(1, (boxes[i][1] * imW)))
              ymax = int(min(imH, (boxes[i][2] * imH)))
              xmax = int(min(imW, (boxes[i][3] * imW)))

              cv2.rectangle(image_rgb, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)

              # Draw label
              object_name = labels[int(classes[i])]
              label = '%s: %d%%' % (object_name, int(scores[i] * 100))
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
              label_ymin = max(ymin, labelSize[1] + 10)
              cv2.rectangle(image_rgb, (xmin, label_ymin - labelSize[1] - 10),
                            (xmin + labelSize[0], label_ymin + baseLine - 10), (255, 255, 255), cv2.FILLED)
              cv2.putText(image_rgb, label, (xmin, label_ymin - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

              detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])
              print(f"detections: {detections}")

      # All the results have been drawn on the image, now display the image
      if not txt_only:
          plt.figure(figsize=(12, 16))
          plt.imshow(image_rgb)
          plt.axis('off')
          plt.show()

      # Save detection results in .txt files (for calculating mAP)
      elif txt_only:
          image_fn = os.path.basename(image_path)
          base_fn, ext = os.path.splitext(image_fn)
          txt_result_fn = base_fn + '.pbtxt'
          txt_savepath = os.path.join(savepath, txt_result_fn)

          with open(txt_savepath, 'w') as f:
              for detection in detections:
                  f.write('%s %.4f %d %d %d %d\n' %
                          (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

  return


In [ ]:
# Set up variables for running user's model
PATH_TO_IMAGES='/content/tst'   # Path to test images folder
PATH_TO_MODEL='/content/custom_model_lite/detect.tflite'   # Path to .tflite model file
PATH_TO_LABELS='/content/sssd2-1/train/label_map.pbtxt'   # Path to labelmap.txt file
min_conf_threshold=0.5   # Confidence threshold (try changing this to 0.01 if you don't see any detection results)
images_to_test = 6   # Number of images to run detection

# Run inferencing function!
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test)

In [ ]:
import tensorflow as tf

model_path = '/content/modelTraffic/content/exported_model/saved_model'
detect_fn = tf.saved_model.load(model_path)
print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
from PIL import Image

result_image = Image.fromarray(image_np)
result_image.save('/content/detected_image.jpg')
print("Saved the result image as detected_image.jpg")


Saved the result image as detected_image.jpg


In [ ]:
!chmod -R 755 static/uploads


In [ ]:
!pip install flask


In [117]:
!python "/content/app.py"


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
YOLO model loaded successfully!
 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
YOLO model loaded successfully!
 * Debugger is active!
 * Debugger PIN: 938-592-970
127.0.0.1 - - [11/Jan/2025 21:48:56] "GET /?authuser=1 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2025 21:48:57] "GET /favicon.ico?authuser=1 HTTP/1.1" 404 -

image 1/1 /content/static/uploads/1.jpg: 544x640 1 Speed limit -20kmh-, 47.2ms
Speed: 3.4ms preprocess, 47.2ms inference, 612.7ms postprocess per image at shape (1, 3, 544, 640)
127.0.0.1 - - [11/Jan/2025 21:49:10] "POST /predict?authuser=1 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2025 21:49:10] "GET /static/uploads/tflite_1.jpg?authuser=1 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2025 21:49:10] "GET /static/uploads/yolo_1.jpg?authuser=1 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jan/2025 21:49:12] "GET

In [116]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))


https://6jxoc2hq3yc-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
!pip install ultralytics

In [ ]:
!roboflow login

visit https://app.roboflow.com/auth-cli to get your authentication token.
Paste the authentication token here: 


In [ ]:
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="V9lggyHp9idDjdLHtQsR")
project = rf.workspace("thesis-0jqhr").project("sssd2")
version = project.version(3)
dataset = version.download("yolov11",location="/content")


loading Roboflow workspace...
loading Roboflow project...


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
!unzip -q "/content/drive/MyDrive/Sravan_Dataset/sssd2.v3i.yolov11.zip" -d /content/dataset_yolov11v2/


In [ ]:
from ultralytics import YOLO

# Load YOLO11n, train it on COCO128 for 3 epochs and predict an image with it
from ultralytics import YOLO

model = YOLO('yolo11n.pt')  # load a pretrained YOLO detection model
DATA_YAML_PATH="/content/dataset_yolov11v2/data.yaml"



# Train model
results = model.train(
    data=DATA_YAML_PATH,          # Dataset config file path
    model=model,             # Pre-trained model path
    epochs=15,                   # Increase epochs to improve convergence
    # time=24,                      # Maximum training time
    patience=25,                  # Early stopping patience
    batch=64,                     # Reduce batch size to fit GPU memory
    imgsz=640,                    # Input image size
    device=0,                     # Use GPU
    workers=10,                    # Dataloader worker threads
    project="/content/yolo_training_logs_v11", # Directory for training logs
    name="/content/yolov8_full_train_v11",     # Experiment name
    exist_ok=True,                # Overwrite existing directory
    pretrained=True,              # Use pretrained weights
    resume=False,                 # Resume from last checkpoint
    amp=True,                     # Enable mixed precision training
    fraction=1.0,                 # Use full dataset

    # # Optimizer and Scheduler
    # optimizer='AdamW',            # AdamW optimizer
    # lr0=0.005,                    # Lower initial learning rate
    # lrf=0.01,                     # Final learning rate fraction
    # momentum=0.937,               # Momentum for SGD
    # weight_decay=0.0005,          # L2 regularization weight decay
    # warmup_epochs=3.0,            # Warmup epochs
    # cos_lr=True,                  # Use cosine scheduler

    # # Data Augmentations
    # hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, # HSV augmentation
    # mosaic=1.0, translate=0.2, scale=0.6, mixup=0.2,  # Stronger augmentations
    # close_mosaic=5,               # Disable mosaic augmentation last 5 epochs

    # # Loss Function Parameters
    # box=7.5, cls=0.5, dfl=1.5,

    # Validation and Regularization
    val=True, save=True, save_period=10, plots=True,
    cache='disk',                 # Cache dataset on disk
    deterministic=True,           # Ensure reproducibility
    # freeze=5,                     # Freeze fewer layers

    # Advanced
    seed=42,                      # Reproducibility
    # max_det=300                   # Max detections per image
)

# Print training results
print("Training complete. Results:", results)


Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, ker

train: Scanning /content/dataset_yolov11v2/train/labels... 6392 images, 2 backgrounds, 0 corrupt: 100%|██████████| 6392/6392 [00:04<00:00, 1416.71it/s]


train: New cache created: /content/dataset_yolov11v2/train/labels.cache


train: Caching images (7.3GB Disk): 100%|██████████| 6392/6392 [00:19<00:00, 325.45it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

      50/50      8.81G     0.3252     0.1654      1.085         57        640: 100%|██████████| 100/100 [01:00<00:00,  1.66it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.55it/s]

                   all        341        341      0.992      0.996      0.995      0.957



50 epochs completed in 0.896 hours.
Optimizer stripped from yolo_training_logs_v11/yolov8_full_train_v11/weights/last.pt, 5.5MB
Optimizer stripped from yolo_training_logs_v11/yolov8_full_train_v11/weights/best.pt, 5.5MB

Validating yolo_training_logs_v11/yolov8_full_train_v11/weights/best.pt...
Ultralytics 8.3.59 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,586,052 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.06s/it]


                   all        341        341       0.99      0.992      0.995      0.959
   Speed limit -20kmh-          6          6      0.985          1      0.995      0.922
     beware of icesnow         16         16      0.995          1      0.995       0.96
     bicycles crossing          6          6      0.965          1      0.995      0.968
            bumpy road          7          7      0.987          1      0.995      0.983
     children crossing         18         18      0.995          1      0.995      0.976
dangerous curve to the left          8          8      0.936          1      0.995      0.977
dangerous curve to the right         12         12          1      0.869      0.995      0.926
          double curve         12         12      0.992          1      0.995      0.977
       general caution         31         31      0.997          1      0.995       0.98
              no entry         16         16      0.996          1      0.995      0.941
          

In [86]:
from ultralytics import YOLO  # For YOLOv8
import torch

# Load your YOLO model (replace with your model path if you have a pretrained model)
model = YOLO('yolo11n.pt')  # load a pretrained YOLO detection model
# Perform training or other operations if needed


# Save the model weights to .pt format
output_path = "/content/best_yolov11_model.pt"  # Specify the output path
torch.save(model.model.state_dict(), output_path)

print(f"Model weights saved as: {output_path}")


Model weights saved as: /content/best_yolov11_model.pt


In [119]:
!zip -r /content/templates.zip /content/templates
!zip -r /content/static.zip /content/static
!zip -r /content/custom_model_lite.zip /content/custom_model_lite
!zip -r /content/yolov11.zip /content/yolov8_full_train_v11

  adding: content/templates/ (stored 0%)
  adding: content/templates/index2.html (deflated 67%)
  adding: content/templates/.ipynb_checkpoints/ (stored 0%)
  adding: content/templates/index.html (deflated 71%)
  adding: content/static/ (stored 0%)
  adding: content/static/.ipynb_checkpoints/ (stored 0%)
  adding: content/static/uploads/ (stored 0%)
  adding: content/static/uploads/result_gettyimages-187499502-612x612.jpg (deflated 19%)
  adding: content/static/uploads/bogen-germany-15th-dec-2022-a-traffic-sign-with-a-snowflake-warning-of-slippery-roads-stands-at-the-side-of-the-road-black-ice-continues-to-plague-parts-of-southern-germany-cred.jpg (deflated 1%)
  adding: content/static/uploads/result_traffic-sign-priority-road-600w-1303931167.webp (deflated 0%)
  adding: content/static/uploads/result_german-road-sign-dangerous-curve-260nw-369163373.webp (deflated 0%)
  adding: content/static/uploads/result_bogen-germany-15th-dec-2022-a-traffic-sign-with-a-snowflake-warning-of-slippery-r